In [1]:
import os
os.environ["GOOGLE_API_KEY"]=""

In [2]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [2]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [3]:
!pip install yt-dlp

In [18]:
!yt-dlp --write-sub --skip-download --sub-lang "en,en-US" "https://www.youtube.com/watch?v=aDG1T0kJnd4"


[youtube] Extracting URL: https://www.youtube.com/watch?v=aDG1T0kJnd4
[youtube] aDG1T0kJnd4: Downloading webpage
[youtube] aDG1T0kJnd4: Downloading tv client config
[youtube] aDG1T0kJnd4: Downloading tv player API JSON
[youtube] aDG1T0kJnd4: Downloading ios player API JSON
[youtube] aDG1T0kJnd4: Downloading m3u8 information
[info] aDG1T0kJnd4: Downloading subtitles: en
[info] aDG1T0kJnd4: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: ENGLISH SPEECH ｜ PRIYANKA CHOPRA： Be Fearless (English Subtitles) [aDG1T0kJnd4].en.vtt
[download] Destination: ENGLISH SPEECH ｜ PRIYANKA CHOPRA： Be Fearless (English Subtitles) [aDG1T0kJnd4].en.vtt
[download] 100% of   21.21KiB in 00:00:00 at 499.08KiB/s


In [4]:
from pathlib import Path

def vtt_to_text(path):
    lines = Path(path).read_text(encoding='utf-8').splitlines()
    text = []
    for line in lines:
        if line and not line.startswith(("WEBVTT", "NOTE")) and "-->" not in line and not line.strip().isdigit():
            text.append(line)
    return " ".join(text)

# Replace with your actual filename
print(vtt_to_text("/content/ENGLISH SPEECH ｜ PRIYANKA CHOPRA： Be Fearless (English Subtitles) [aDG1T0kJnd4].en.vtt"))


Kind: captions Language: en Thank you. Thank you. So, my father used to always tell me something which I want to share with you, that why do you want to fit inside a glass slipper? You know, like we were told, like Cinderella did, why do you want to fit inside a glass slipper when you can shatter the glass ceiling? I want to tell you a little secret. I'm not very fond of this phrase, breaking the glass ceiling. Why does it annoy me? Because it takes the context of everything that I have done. All my achievements, all my hard work and puts it into a box as if my ambition was that I want to find a glass ceiling and break it. Not at all. To be really honest, I was never on a mission to break, to shatter anything. All I wanted was to chase my dreams, my ambitions. I wanted to evolve. I wanted to become the best version of me, that I could be. And of course, in Priyanka Chopra style, along the way, I wanted to smash and break every obstacle that came my way, which I did. But that does not m

In [5]:
import os

# List files in the current directory
for f in os.listdir():
    if f.endswith(".vtt"):
        print(f)


ENGLISH SPEECH ｜ PRIYANKA CHOPRA： Be Fearless (English Subtitles) [aDG1T0kJnd4].en.vtt


In [6]:
from pathlib import Path

def vtt_to_text(vtt_path):
    lines = Path(vtt_path).read_text(encoding='utf-8').splitlines()
    text = []
    for line in lines:
        # Skip metadata, timestamps, and indexes
        if line and not line.startswith(("WEBVTT", "NOTE", "STYLE")) \
                and "-->" not in line and not line.strip().isdigit():
            text.append(line.strip())
    return " ".join(text)

# Replace with your actual subtitle filename
transcript_text = vtt_to_text("/content/ENGLISH SPEECH ｜ PRIYANKA CHOPRA： Be Fearless (English Subtitles) [aDG1T0kJnd4].en.vtt")

# Print or use it however you want
print(transcript_text[:10000])  # Print first 1000 characters


Kind: captions Language: en Thank you. Thank you. So, my father used to always tell me something which I want to share with you, that why do you want to fit inside a glass slipper? You know, like we were told, like Cinderella did, why do you want to fit inside a glass slipper when you can shatter the glass ceiling? I want to tell you a little secret. I'm not very fond of this phrase, breaking the glass ceiling. Why does it annoy me? Because it takes the context of everything that I have done. All my achievements, all my hard work and puts it into a box as if my ambition was that I want to find a glass ceiling and break it. Not at all. To be really honest, I was never on a mission to break, to shatter anything. All I wanted was to chase my dreams, my ambitions. I wanted to evolve. I wanted to become the best version of me, that I could be. And of course, in Priyanka Chopra style, along the way, I wanted to smash and break every obstacle that came my way, which I did. But that does not m

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript_text])

# Preview a few chunks
for i, chunk in enumerate(chunks[:3]):
    print(f"\n--- Chunk {i+1} ---\n{chunk.page_content}")



--- Chunk 1 ---
Kind: captions Language: en Thank you. Thank you. So, my father used to always tell me something which I want to share with you, that why do you want to fit inside a glass slipper? You know, like we were told, like Cinderella did, why do you want to fit inside a glass slipper when you can shatter the glass ceiling? I want to tell you a little secret. I'm not very fond of this phrase, breaking the glass ceiling. Why does it annoy me? Because it takes the context of everything that I have done. All my achievements, all my hard work and puts it into a box as if my ambition was that I want to find a glass ceiling and break it. Not at all. To be really honest, I was never on a mission to break, to shatter anything. All I wanted was to chase my dreams, my ambitions. I wanted to evolve. I wanted to become the best version of me, that I could be. And of course, in Priyanka Chopra style, along the way, I wanted to smash and break every obstacle that came my way, which I did. Bu

In [8]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# 1. Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True)[0].tolist()


In [10]:
embedding_model = SentenceTransformerEmbeddings('all-MiniLM-L6-v2')  # or any other supported model

from langchain.vectorstores import FAISS
vector_store = FAISS.from_documents(chunks, embedding=embedding_model)


In [11]:
vector_store.index_to_docstore_id

{0: '21db6d88-6508-4774-b268-d7fcab3d15af',
 1: '53ed9253-b939-4406-a469-6b94e2728554',
 2: '81d80bc8-9925-4490-8dda-356dbc2c5b97',
 3: 'ac3eab07-a63b-40aa-9fd1-b6f9c32fce60',
 4: '54ee504e-e558-497b-a195-e361abe1de15',
 5: 'ad4ae102-e3bc-4ce7-a6e7-4a0e49e3b07f',
 6: '69acc9c3-7c5e-447f-abea-b2c6766bf321',
 7: 'b398119e-3372-4641-ae49-d58c4da9c56d',
 8: 'e6bf0c76-eddd-4574-9a63-c8640582909c',
 9: '5eb1e4f1-fb4d-4008-a7e3-d36d42791db2',
 10: '2e2dccf9-a223-4cc3-90df-f8a173785dba',
 11: '146af237-4de5-4034-854f-6b4f6c843703',
 12: 'fcd70f98-8d7a-458b-b320-b342e06ae41a',
 13: '55b09372-fe20-4302-904b-74a688c00996',
 14: 'b5eae6b5-54e1-47fa-af12-d89afb7bfe4c',
 15: 'd4cd8a49-59d9-4d51-a6ed-969fb2412848',
 16: '82735289-005f-49db-93c2-47c95bbec409'}

In [15]:
vector_store.get_by_ids(['82735289-005f-49db-93c2-47c95bbec409'])

[Document(id='82735289-005f-49db-93c2-47c95bbec409', metadata={}, page_content="not the least, rule number twelve, don't ever forget your roots and where you came from. I hope you take some of what I have shared with you today and go into the new year with maybe a different perspective on life and start 2018 with being fierce, fearless, and flawed. Yeah, that's it. That's it. I read this somewhere and it kind of stuck with me and I hope it will stick with you. Life's journey is not to arrive at your grave site safely, in a well-preserved body, but rather to skid in sideways, completely worn out and say, holy shit, that was a ride. That is how I want to go. So, to sum up this incredible journey that I have been on, I stand before you today, weathered by life, but happy and raring to do so much more. I'll see you on the other side. Namaste.")]

In [16]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})


In [17]:
retriever

VectorStoreRetriever(tags=['FAISS', 'SentenceTransformerEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fe14720e350>, search_kwargs={'k': 4})

In [18]:
retriever.invoke('What did Priyanka Chopra say about fear?')

[Document(id='21db6d88-6508-4774-b268-d7fcab3d15af', metadata={}, page_content="Kind: captions Language: en Thank you. Thank you. So, my father used to always tell me something which I want to share with you, that why do you want to fit inside a glass slipper? You know, like we were told, like Cinderella did, why do you want to fit inside a glass slipper when you can shatter the glass ceiling? I want to tell you a little secret. I'm not very fond of this phrase, breaking the glass ceiling. Why does it annoy me? Because it takes the context of everything that I have done. All my achievements, all my hard work and puts it into a box as if my ambition was that I want to find a glass ceiling and break it. Not at all. To be really honest, I was never on a mission to break, to shatter anything. All I wanted was to chase my dreams, my ambitions. I wanted to evolve. I wanted to become the best version of me, that I could be. And of course, in Priyanka Chopra style, along the way, I wanted to s

In [19]:
query = "What did Priyanka Chopra say about fear?"
results = retriever.get_relevant_documents(query)

for i, doc in enumerate(results, 1):
    print(f"--- Result {i} ---\n{doc.page_content}\n")


--- Result 1 ---
Kind: captions Language: en Thank you. Thank you. So, my father used to always tell me something which I want to share with you, that why do you want to fit inside a glass slipper? You know, like we were told, like Cinderella did, why do you want to fit inside a glass slipper when you can shatter the glass ceiling? I want to tell you a little secret. I'm not very fond of this phrase, breaking the glass ceiling. Why does it annoy me? Because it takes the context of everything that I have done. All my achievements, all my hard work and puts it into a box as if my ambition was that I want to find a glass ceiling and break it. Not at all. To be really honest, I was never on a mission to break, to shatter anything. All I wanted was to chase my dreams, my ambitions. I wanted to evolve. I wanted to become the best version of me, that I could be. And of course, in Priyanka Chopra style, along the way, I wanted to smash and break every obstacle that came my way, which I did. Bu

<ipython-input-19-1906352377>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [20]:
!pip install langchain-google-genai


In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # or "gemini-pro" if you prefer
    temperature=0.2
)


In [22]:
from langchain.prompts import PromptTemplate


In [23]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [24]:
question = "What is the main message or theme of Priyanka Chopra's 'Be Fearless' speech?"
retrieved_docs    = retriever.invoke(question)

In [25]:
retrieved_docs

[Document(id='81d80bc8-9925-4490-8dda-356dbc2c5b97', metadata={}, page_content="if I hated it so much? Because I hope through my experiences, I can present to you the tools to live your dream and to not make that glass ceiling that is defined by society, largely patriarchal, as your goal. I want you to aim higher, because you know you can, and it's honestly all I did. Every step of the way, all I did was set a higher standard. How did I get here? By being fierce, by being fearless, and by being flawed, completely and utterly flawed. I'm not perfect. Like, none of us are. So, tonight I'm going to try and break it down for you a little bit in a slightly different way. I present the twelve sure shot, sure fireways of becoming Priyanka Chopra. Would you like that? Sorry, I was just kidding. Because as millennials, we don't like to be told what to do. So, this is a gentle nudge in the right direction. Presenting PCs twelve rules of becoming the best version of yourself. So, here are a few o

In [26]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"if I hated it so much? Because I hope through my experiences, I can present to you the tools to live your dream and to not make that glass ceiling that is defined by society, largely patriarchal, as your goal. I want you to aim higher, because you know you can, and it's honestly all I did. Every step of the way, all I did was set a higher standard. How did I get here? By being fierce, by being fearless, and by being flawed, completely and utterly flawed. I'm not perfect. Like, none of us are. So, tonight I'm going to try and break it down for you a little bit in a slightly different way. I present the twelve sure shot, sure fireways of becoming Priyanka Chopra. Would you like that? Sorry, I was just kidding. Because as millennials, we don't like to be told what to do. So, this is a gentle nudge in the right direction. Presenting PCs twelve rules of becoming the best version of yourself. So, here are a few of my favorites from my list. There's only one you, okay, because there's no one

In [27]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [28]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      if I hated it so much? Because I hope through my experiences, I can present to you the tools to live your dream and to not make that glass ceiling that is defined by society, largely patriarchal, as your goal. I want you to aim higher, because you know you can, and it's honestly all I did. Every step of the way, all I did was set a higher standard. How did I get here? By being fierce, by being fearless, and by being flawed, completely and utterly flawed. I'm not perfect. Like, none of us are. So, tonight I'm going to try and break it down for you a little bit in a slightly different way. I present the twelve sure shot, sure fireways of becoming Priyanka Chopra. Would you like that? Sorry, I was just kidding. Because as millennials, we don't like to be told what to do. So, this is a gentle nudge in 

In [29]:
answer = llm.invoke(final_prompt)
print(answer.content)

Priyanka Chopra's speech focuses on aiming higher than the societal "glass ceiling,"  and becoming the best version of yourself.  She emphasizes chasing dreams and ambitions, smashing obstacles along the way, but not defining success solely by breaking the glass ceiling.  She shares her personal experiences and offers "twelve rules" (though only a few are detailed) to help others achieve their goals.


In [30]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [31]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [32]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [34]:
parallel_chain.invoke('What career or life advice does Priyanka Chopra give in the speech?')

{'context': "Kind: captions Language: en Thank you. Thank you. So, my father used to always tell me something which I want to share with you, that why do you want to fit inside a glass slipper? You know, like we were told, like Cinderella did, why do you want to fit inside a glass slipper when you can shatter the glass ceiling? I want to tell you a little secret. I'm not very fond of this phrase, breaking the glass ceiling. Why does it annoy me? Because it takes the context of everything that I have done. All my achievements, all my hard work and puts it into a box as if my ambition was that I want to find a glass ceiling and break it. Not at all. To be really honest, I was never on a mission to break, to shatter anything. All I wanted was to chase my dreams, my ambitions. I wanted to evolve. I wanted to become the best version of me, that I could be. And of course, in Priyanka Chopra style, along the way, I wanted to smash and break every obstacle that came my way, which I did. But th

In [35]:
parser = StrOutputParser()


In [36]:
main_chain = parallel_chain | prompt | llm | parser

In [37]:
main_chain.invoke('Can you summarize the video')

'The speaker, Priyanka Chopra, discusses her dislike of the phrase "breaking the glass ceiling," preferring to focus on chasing dreams and evolving into the best version of herself.  She shares her "twelve rules of becoming the best version of yourself," highlighting the importance of setting high standards, being fierce and fearless, and embracing failure as a learning opportunity.  She emphasizes that failure is inevitable but how one responds to it defines their path.  She describes her own process of dealing with failure, including self-pity followed by getting up and moving forward.  Ultimately, she encourages the audience to aim higher and not compromise their dreams.'